In [27]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from scipy.sparse import hstack, csr_matrix
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import os 
import mlflow

from ML_Utils import *

def predict():

    df = pd.read_csv("/data1/notebooks/H_Level/ML/Data/books_task.csv")
    df_tranformed = data_validation(df)
    os.chdir("/data1")

    mlflow.set_tracking_uri("...")
    # Please put your tracking uri. I have removed per privacy concerns:

    experiment_name = "Book_Impact_Prediction_1"

    runs = mlflow.search_runs(experiment_ids=mlflow.get_experiment_by_name(experiment_name).experiment_id)

    runs = runs.sort_values(['metrics.Mean Absolute Percentage Error'], ascending = [False]).reset_index().drop(['index'], axis = 1)

    best_run = runs.loc[runs["metrics.Mean Absolute Percentage Error"].idxmax()]

    best_model = mlflow.pyfunc.load_model(best_run.artifact_uri + "/model")

    predicted = best_model.predict(df_tranformed)

    with open('/data1/notebooks/H_Level/ML/Scaler Pickle/scaler.pkl', 'rb') as f:
        loaded_scaler = pickle.load(f)

    # Use the loaded scaler to transform data
    predicted = loaded_scaler.inverse_transform(pd.DataFrame(predicted))


    df['Book_Impact'] = pd.DataFrame(predicted)
    
    return df

def data_validation(df):
    
    # Pre Processing of Data

    data_preprocessed = general_preprocessing(df)
    data_preprocessed.removesquarebracket(columns = ['authors', 'categories'])
    data_preprocessed.date_processing(columns = ['publishedDate'])
    data_preprocessed.dropna()

    df_tranformed = TextVectorizer(64)
    df_tranformed = df_tranformed.fit_transform(df, text_columns = ['Title', 'description', 'authors', 'publisher', 'categories'], numerical_column = ['publishedDate'])

    return df_tranformed


In [28]:
predict()

/data1/notebooks/H_Level/ML/Code/ML_Utils.py:64: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  self.data[col] = self.data[col].str.replace("[", "").str.replace("]", "").str.replace("'", "")


,Unnamed: 0,Title,description,authors,publisher,publishedDate,categories,Impact,Book_Impact
0,1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,Philip Nel,A&C Black,2005,Biography & Autobiography,825.465535,813.049011
1,2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,David R. Ray,OUP USA,2000,Religion,841.705321,844.613647
2,3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,Veronica Haddon,iUniverse,2005,Fiction,666.426542,770.839600
3,5,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,Everett Ferguson,Wm. B. Eerdmans Publishing,1996,Religion,806.216143,795.337830
4,8,Saint Hyacinth of Poland,The story for children 10 and up of St. Hyacin...,Mary Fabyan Windeatt,Tan Books & Pub,2009,Biography & Autobiography,799.162610,805.875122
...,...,...,...,...,...,...,...,...,...
124393,212394,Final things,Grace's father believes in science and builds ...,Jenny Offill,Vintage,2015,Fiction,784.303924,754.327332
124394,212397,The Magic of the Soul: Applying Spiritual Powe...,"""The Magic of the Soul, Applying Spiritual Pow...",Patrick J. Harbula,Love & Logic Press,2002,"Body, Mind & Spirit",897.130211,804.116089
124395,212398,Autodesk Inventor 10 Essentials Plus,Autodesk Inventor 2017 Essentials Plus provide...,"Daniel Banach, Travis Jones",SDC Publications,2016,Computers,827.848586,769.761108
124396,212399,The Orphan Of Ellis Island (Time Travel Advent...,"During a school trip to Ellis Island, Dominick...",Elvira Woodruff,Scholastic Paperbacks,2000,Juvenile Fiction,876.786309,803.174927
